In [5]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage
from typing import List, Annotated, TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import pandas as pd
import os
import ast

_ = load_dotenv()
model = ChatOpenAI(model="gpt-4o", temperature=0.2)


In [6]:
import pandas as pd

df = pd.read_csv('/Users/leopoldotodisco/Desktop/progetto-SAD/Phishing-Legitimate-Analisys/codice_nuovo/Phishing_Legitimate_full.csv')

for col in df.columns:
    min_val = df[col].min()
    max_val = df[col].max()
    print(f'Colonna: {col} -> Min: {min_val}, Max: {max_val}')


FileNotFoundError: [Errno 2] No such file or directory: '/Users/leopoldotodisco/Desktop/progetto-SAD/Phishing-Legitimate-Analisys/Phishing_Legitimate_full.csv'

In [85]:
class AgentState(TypedDict):
    messages: Annotated[list[dict], operator.add]
    risultati_generati: dict
    critica: str
    max_revisions: int
    revision_number: int

In [86]:
expected_columns = {
    "id": {"type": int, "min": 0, "max": None},
    "NumDots": {"type": int, "min": 0, "max": None},
    "SubdomainLevel": {"type": int, "min": 0, "max": None},
    "PathLevel": {"type": int, "min": 0, "max": None},
    "UrlLength": {"type": int, "min": 0, "max": None},
    "NumDash": {"type": int, "min": 0, "max": None},
    "NumDashInHostname": {"type": int, "min": 0, "max": None},
    "AtSymbol": {"type": int, "min": 0, "max": 1},
    "TildeSymbol": {"type": int, "min": 0, "max": 1},
    "NumUnderscore": {"type": int, "min": 0, "max": None},
    "NumPercent": {"type": int, "min": 0, "max": None},
    "NumQueryComponents": {"type": int, "min": 0, "max": None},
    "NumAmpersand": {"type": int, "min": 0, "max": None},
    "NumHash": {"type": int, "min": 0, "max": None},
    "NumNumericChars": {"type": int, "min": 0, "max": None},
    "NoHttps": {"type": int, "min": 0, "max": 1},
    "RandomString": {"type": int, "min": 0, "max": 1},
    "IpAddress": {"type": int, "min": 0, "max": 1},
    "DomainInSubdomains": {"type": int, "min": 0, "max": 1},
    "DomainInPaths": {"type": int, "min": 0, "max": 1},
    "HttpsInHostname": {"type": int, "min": 0, "max": 0},
    "HostnameLength": {"type": int, "min": 0, "max": None},
    "PathLength": {"type": int, "min": 0, "max": None},
    "QueryLength": {"type": int, "min": 0, "max": None},
    "DoubleSlashInPath": {"type": int, "min": 0, "max": 1},
    "NumSensitiveWords": {"type": int, "min": 0, "max": None},
    "EmbeddedBrandName": {"type": int, "min": 0, "max": 1},
    "PctExtHyperlinks": {"type": float, "min": 0.0, "max": 1.0},
    "PctExtResourceUrls": {"type": float, "min": 0.0, "max": 1.0},
    "ExtFavicon": {"type": int, "min": 0, "max": 1},
    "InsecureForms": {"type": int, "min": 0, "max": 1},
    "RelativeFormAction": {"type": int, "min": 0, "max": 1},
    "ExtFormAction": {"type": int, "min": 0, "max": 1},
    "AbnormalFormAction": {"type": int, "min": 0, "max": 1},
    "PctNullSelfRedirectHyperlinks": {"type": float, "min": 0.0, "max": 1.0},
    "FrequentDomainNameMismatch": {"type": int, "min": 0, "max": 1},
    "FakeLinkInStatusBar": {"type": int, "min": 0, "max": 1},
    "RightClickDisabled": {"type": int, "min": 0, "max": 1},
    "PopUpWindow": {"type": int, "min": 0, "max": 1},
    "SubmitInfoToEmail": {"type": int, "min": 0, "max": 1},
    "IframeOrFrame": {"type": int, "min": 0, "max": 1},
    "MissingTitle": {"type": int, "min": 0, "max": 1},
    "ImagesOnlyInForm": {"type": int, "min": 0, "max": 1},
    "SubdomainLevelRT": {"type": int, "min": -1, "max": 1},
    "UrlLengthRT": {"type": int, "min": -1, "max": 1},
    "PctExtResourceUrlsRT": {"type": float, "min": -1, "max": 1.0},
    "AbnormalExtFormActionR": {"type": int, "min": -1, "max": 1},
    "ExtMetaScriptLinkRT": {"type": int, "min": -1, "max": 1},
    "PctExtNullSelfRedirectHyperlinksRT": {"type": float, "min": -1, "max": 1},
    "CLASS_LABEL": {"type": int, "min": 0, "max": 1},
}

In [87]:
GENERATOR_PROMPT = """You are a data generator. 
Your task is to generate 1 sample based on the information I will provide. 
The dataset you must generate is about phishing and legit websites. The columns are:

id, NumDots, SubdomainLevel, PathLevel, UrlLength, NumDash, NumDashInHostname, AtSymbol, TildeSymbol, NumUnderscore, NumPercent, NumQueryComponents, NumAmpersand, NumHash, NumNumericChars, NoHttps, RandomString, IpAddress, DomainInSubdomains, DomainInPaths, HttpsInHostname, HostnameLength, PathLength, QueryLength, DoubleSlashInPath, NumSensitiveWords, EmbeddedBrandName, PctExtHyperlinks, PctExtResourceUrls, ExtFavicon, InsecureForms, RelativeFormAction, ExtFormAction, AbnormalFormAction, PctNullSelfRedirectHyperlinks, FrequentDomainNameMismatch, FakeLinkInStatusBar, RightClickDisabled, PopUpWindow, SubmitInfoToEmail, IframeOrFrame, MissingTitle, ImagesOnlyInForm, SubdomainLevelRT, UrlLengthRT, PctExtResourceUrlsRT, AbnormalExtFormActionR, ExtMetaScriptLinkRT, PctExtNullSelfRedirectHyperlinksRT, CLASS_LABEL

Examples are:
2, 3, 1, 3, 144, 0, 0, 0, 0, 2, 0, 2, 1, 0, 41, 1, 0, 0, 0, 0, 0, 17, 16, 103, 0, 1, 0, 0.0000000000, 0.0000000000, 0, 1, 0, 0, 0, 0.0000000000, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 1, 1, 1, 1, 1
6434, 3, 1, 1, 143, 10, 0, 0, 0, 0, 0, 0, 0, 0, 5, 1, 1, 0, 0, 1, 0, 42, 94, 0, 0, 0, 0, 0.1250000000, 0.2093023256, 0, 0, 0, 0, 0, 0.2000000000, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 1, 1, 0, 0, 0

You must respond with data only, without any introductory or explanatory text. 
The structure must be a dictionary-like format because I will use `ast.literal_eval` to parse the data you generate.
"""

def generator_node(state: AgentState):
    messages = [
        SystemMessage(content = GENERATOR_PROMPT),
        HumanMessage(content=f"These are the rules you must follow: {repr(expected_columns)}")
    ]
    
    response = model.invoke(messages)

    return {"risultati_generati": ast.literal_eval(response.content)}

In [88]:
def check_data_entry(data, expected_columns):
    """
    Verifica i tipi di dato e i range di valori per un dizionario `data`.

    :param data: Dizionario contenente i dati da verificare.
    :param expected_columns: Dizionario che definisce i tipi e i range attesi per ogni colonna.
    :return: Lista di problemi riscontrati.
    """
    issues = []
    
    for column, details in expected_columns.items():
        if column not in data:
            issues.append({
                "Column": column,
                "Issue": "Column Missing",
                "Actual Value": None,
                "Expected Type": details["type"]
            })
            continue
        
        value = data[column]
        
        if not isinstance(value, details["type"]):
            issues.append({
                "Column": column,
                "Issue": "Type Mismatch",
                "Actual Value": type(value).__name__,
                "Expected Type": details["type"].__name__
            })
        
        if details["min"] is not None and value < details["min"]:
            issues.append({
                "Column": column,
                "Issue": "Value Below Minimum",
                "Actual Value": value,
                "Expected Min": details["min"]
            })
            
        if details["max"] is not None and value > details["max"]:
            issues.append({
                "Column": column,
                "Issue": "Value Above Maximum",
                "Actual Value": value,
                "Expected Max": details["max"]
            })
            
    return issues

def critique_node(state: AgentState):
    critica = "OK"
    data = state["risultati_generati"]
    issues = check_data_entry(data, expected_columns)
    issues = False
    if issues:
        critica = "Generated data values are not within the limits"
    else:
        critica = "OK"
        
    return {"critica": critica}

In [89]:
def should_continue(state: AgentState):
    if state["critica"] == "OK":
        return True
    
    if state["revision_number"] > state["max_revisions"] or state["critica"] != "OK":
        return False # torna al nodo di generazione

In [3]:
builder = StateGraph(AgentState)

builder.add_node("generator_node", generator_node)
builder.add_node("critique_node", critique_node)
builder.add_edge("generator_node", "critique_node")
builder.add_conditional_edges("critique_node", should_continue, {True: END, False: "generator_node"})

builder.set_entry_point("generator_node")

graph = builder.compile()

NameError: name 'StateGraph' is not defined

In [2]:
from IPython.display import Image

Image(graph.get_graph().draw_png())

NameError: name 'graph' is not defined

In [1]:
# Genera il grafo in formato PNG e lo salva come file
graph.get_graph().draw("grafo.png", format="png")

# Mostra l'immagine salvata
Image("grafo.png")

NameError: name 'graph' is not defined

In [ ]:
import pandas as pd

risultati = []

for j in range(10):
    for i in range(1000):
        print(f"""elemento: {i}""")
        try:
            response = graph.invoke(
                {"critica": "",
                "max_revisions": 4,
                "revision_number": 0}
            )
        except Exception as e:
            print("eccezione")    
            
        risultati.append(None if not response["risultati_generati"] else response["risultati_generati"])
        
    df = pd.DataFrame()

    for r in risultati:
        temp_df = pd.DataFrame([r])
        
        df = pd.concat([df, temp_df], ignore_index=True)
    
df.to_csv("../data/dataset.csv", index=False)

## Controllo dei duplicati

In [155]:
df = pd.read_csv("../data/dataset.csv")
duplicates = df[df.duplicated()]

duplicates.shape[0]

0

In [154]:
duplicates.size

0